- MSSV: 21120149
- Họ và tên: Nguyễn Đăng Thới Toàn

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline


In [2]:
#Em lấy down file csv của thầy rồi up lên Github
url='https://raw.githubusercontent.com/keiranoluv/data/main/weatherPrediction_regression.csv'

#Đọc dữ liệu
data = pd.read_csv(url)
data


table = []

# 1. Decision Tree

**a) Xem mỗi cột dữ liệu là kiểu dữ liệu nominal => Sử dụng One Hot Encoding**

In [3]:
#Xem như 4 features của data là dữ liệu không có thứ tự
categorical_cols = ['Outlook', 'Temp', 'Humidity', 'Windy']

X = data[['Outlook', 'Temp', 'Humidity', 'Windy']]  #  Input
y = data['Hours Played']  #Output

# Tiền xử lý
preprocessor = ColumnTransformer( transformers=[('cat', OneHotEncoder(), categorical_cols)],
                                remainder='passthrough')

X_prep = preprocessor.fit_transform(X)

# Chia thành 2 tập train test
X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.2, random_state=42)

# Train
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

table.append(["Decision Tree","Nominal",mse,r2])

compare_df = data.loc[y_test.index]
compare_df['Predicted']=y_pred
compare_df

Mean Squared Error: 196.0
R-squared: -0.4626865671641791


,Outlook,Temp,Humidity,Windy,Hours Played,Predicted
9,Sunny,Mild,Normal,False,46,44.0
11,Overcast,Mild,High,True,52,30.0
0,Rainy,Hot,High,False,25,35.0


**b) Xem mỗi cột dữ liệu là kiểu dữ liệu ordinal => Sử dụng Label Encoding**

In [4]:
df = data.copy() 

#Sử dụng Label Encoding vào quá trình tiền xử lý
label_encoder = LabelEncoder()

for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

X = df[categorical_cols]  #Input
y = df['Hours Played']  #Output

# Chia tập train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
table.append(["Decision Tree","Ordinal",mse,r2])

compare_df = data.loc[y_test.index]
compare_df['Predicted']=y_pred
compare_df


Mean Squared Error: 218.0
R-squared: -0.6268656716417911


,Outlook,Temp,Humidity,Windy,Hours Played,Predicted
9,Sunny,Mild,Normal,False,46,45.0
11,Overcast,Mild,High,True,52,30.0
0,Rainy,Hot,High,False,25,38.0


# 2. SVM

**a) Xem mỗi cột dữ liệu là kiểu dữ liệu nominal => Sử dụng One Hot Encoding**

In [5]:
#SVM

df = data.copy()

X = df[categorical_cols] 
y = df['Hours Played']  

#Chia tập train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tiền xử lý và cấu hình model

# Sử dụng One Hot Encoding
preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(), categorical_cols)])                
model = SVR(kernel='rbf')  
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#Train
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

table.append(["SVM","Nominal",mse,r2])

compare_df = data.loc[y_test.index]
compare_df['Predicted']=y_pred
compare_df

Mean Squared Error: 134.7650483115024
R-squared: -0.005709315757480571


,Outlook,Temp,Humidity,Windy,Hours Played,Predicted
9,Sunny,Mild,Normal,False,46,43.318900
11,Overcast,Mild,High,True,52,42.595327
0,Rainy,Hot,High,False,25,42.568693


**b) Xem mỗi cột dữ liệu là kiểu dữ liệu ordinal => Sử dụng Label Encoding**

In [6]:
#SVM

df = data.copy()
#Sử dụng Label Encoding vào quá trình tiền xử lý
label_encoder = LabelEncoder()

for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

X = df[categorical_cols]  #Input
y = df['Hours Played']  #Output
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train
model = SVR(kernel='rbf')  # Radial Basis Function kernel is commonly used
model.fit(X_train_scaled, y_train)

# Dự đoán
y_pred = model.predict(X_test_scaled)

# Đánh giá
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
table.append(["SVM","Ordinal",mse,r2])

compare_df = data.loc[y_test.index]
compare_df['Predicted']=y_pred
compare_df

Mean Squared Error: 140.04620262190062
R-squared: -0.045120915088810554


,Outlook,Temp,Humidity,Windy,Hours Played,Predicted
9,Sunny,Mild,Normal,False,46,43.323767
11,Overcast,Mild,High,True,52,42.680739
0,Rainy,Hot,High,False,25,43.059008


# 3. So sánh hiệu quả Decision Tree và SVM trong bài toán hồi quy

- Độ chính xác:
    - SVM thường có độ chính xác cao hơn Decision Tree, đặc biệt là với dữ liệu phức tạp và không phân tách tuyến tính. Tuy nhiên, hiệu quả của Decision Tree có thể được cải thiện bằng cách sử dụng các kỹ thuật như ensemble learning (ví dụ: Random Forest). Trong mô hình được train ở trên, dễ thấy rằng SVM có MSE thấp hơn Decision Tree.
- Tốc độ:
    - Decision Tree thường nhanh hơn SVM trong quá trình huấn luyện và dự đoán. Tuy nhiên, tốc độ của SVM có thể được cải thiện bằng cách sử dụng các kỹ thuật tối ưu hóa.
- Khả năng giải thích:
    - Decision Tree dễ hiểu và dễ diễn giải hơn SVM.
    - SVM có thể khó hiểu và khó diễn giải do kết quả dự đoán là một hàm toán học phức tạp.
- Khả năng chống overfitting:
    - SVM có khả năng chống overfitting tốt hơn Decision Tree.
    - Decision Tree có thể dễ bị overfitting nếu không được điều chỉnh phù hợp.
- Khả năng xử lý dữ liệu nhiễu:
    - Decision Tree có khả năng xử lý dữ liệu nhiễu tốt hơn SVM.
    - SVM có thể nhạy cảm với dữ liệu nhiễu.

In [7]:
pd.DataFrame(table).set_index([0,1]).rename(columns = {2: "MSE", 3:"R2"})

#Trong một số trường hợp khi chạy thuật toán Decision Tree thì MSE của Decision Tree rất nhỏ, chỉ tầm 40

MSE        R2
0             1                            
Decision Tree Nominal  196.000000 -0.462687
              Ordinal  218.000000 -0.626866
SVM           Nominal  134.765048 -0.005709
              Ordinal  140.046203 -0.045121